In [2]:
import pickle
import numpy as np
import gc

np.random.seed(1234)

In [3]:
'''读原始MDP文件'''

MDP_filename = "../data/raw_MDP.pkl"
with open(MDP_filename, "rb") as f:
    data = pickle.load(f)

MDP_w_tmp = data['MDP_w']
MDP_r_tmp = data['MDP_r']
del data
gc.collect()
print(len(MDP_w_tmp), len(MDP_r_tmp)==len(MDP_w_tmp))

sample_order = np.arange(len(MDP_w_tmp))
np.random.shuffle(sample_order)
sample_order = sample_order.tolist()

2608543 True


In [4]:
FORMER_E_CNT_THRESHOLD = 5
LATTER_E_CNT_THRESHOLD = 2
MAX_AVAILABLE_ACTION = 83

MDP_w, MDP_r = [], []

raw_award_list = []
raw_industry_list = []
raw_category_list = []
raw_project_quality_list = []
raw_start_gap_list = []
raw_end_gap_list = []
for i in range(len(MDP_w_tmp)):
    MDP_w.append(MDP_w_tmp[sample_order[i]])  # 构成新的顺序
    MDP_r.append(MDP_r_tmp[sample_order[i]])  # 构成新的顺序
    if i % 100000 == 0:
        print(i)
    row_state_list = MDP_w_tmp[i]['state']
    for row_state in row_state_list:
        raw_award_list.append(row_state['a_award'])
        if row_state['a_industry'] not in raw_industry_list:
            raw_industry_list.append(row_state['a_industry'])
        if row_state['a_category'] not in raw_category_list:
            raw_category_list.append(row_state['a_category'])
        raw_project_quality_list.append(row_state['project_quality'])
        raw_start_gap_list.append(row_state['a_start_gap'])
        raw_end_gap_list.append(row_state['a_end_gap'])
MAX_AWARD = max(raw_award_list)
MIN_AWARD = min(raw_award_list)
MAX_INDUSTRY_VALUE = max(raw_industry_list)
MAX_CATEGORY_VALUE = max(raw_category_list)
MAX_PROJECT_QUALITY = max(raw_project_quality_list)
MIN_PROJECT_QUALITY = min(raw_project_quality_list)
MAX_START_GAP = max(raw_start_gap_list)
MIN_START_GAP = min(raw_start_gap_list)
MAX_END_GAP = max(raw_end_gap_list)
MIN_END_GAP = min(raw_end_gap_list)
print('MAX_AWARD, MIN_AWARD', MAX_AWARD, MIN_AWARD)
print('industry', sorted(raw_industry_list), MAX_INDUSTRY_VALUE)
print('category', sorted(raw_category_list), MAX_CATEGORY_VALUE)
print('MAX_PROJECT_QUALITY, MIN_PROJECT_QUALITY', MAX_PROJECT_QUALITY, MIN_PROJECT_QUALITY)
print('MAX_START_GAP, MIN_START_GAP', MAX_START_GAP, MIN_START_GAP)
print('MAX_END_GAP, MIN_END_GAP', MAX_END_GAP, MIN_END_GAP)

del MDP_w_tmp, MDP_r_tmp, raw_award_list, raw_industry_list, raw_category_list, raw_project_quality_list, raw_start_gap_list,\
    raw_end_gap_list, sample_order
gc.collect()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
MAX_AWARD, MIN_AWARD 4140.0 0.0
industry [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36] 36
category [0, 1, 2, 3, 4, 5, 6] 6
MAX_PROJECT_QUALITY, MIN_PROJECT_QUALITY 10.797708090145798 0.0
MAX_START_GAP, MIN_START_GAP 381 0
MAX_END_GAP, MIN_END_GAP 338 0


0

In [9]:
'''将原始MDP_w和MDP_r经归一化和填补0后转为numpy格式'''

unit = 100000
for partition in range(0, len(MDP_w)//unit+1):
    print("处理分区%d中……" % partition)

    batch_states, batch_actions, batch_rewards_w, batch_rewards_r, batch_next_states_w, batch_next_states_r = [], [], [], [], [], []
    batch_infos, batch_best_infos, batch_next_infos_w, batch_next_infos_r = [], [], [], []
    for i in range(partition * unit, min((partition + 1) * unit, len(MDP_w))):
        if i % 10000 == 0:
            print("处理记录%d/%d……" % (i, len(MDP_w)))
            
        # 【S, Info】
        state_list, info_list = [], []
        for row_state in MDP_w[i]['state']:
            row_state_list = []
            row_state_list.append(row_state['a_category'])
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                row_state_list.append(row_state['l'+str(j)+'_category'])
            
            row_state_list.append(row_state['a_industry'])
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                row_state_list.append(row_state['l'+str(j)+'_industry'])
            
            row_state_list.append((row_state['a_award']-MIN_AWARD)/(MAX_AWARD-MIN_AWARD))
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                row_state_list.append((row_state['l'+str(j)+'_award']-MIN_AWARD)/(MAX_AWARD-MIN_AWARD))
            
            row_state_list.append((row_state['a_start_gap']-MIN_START_GAP)/(MAX_START_GAP-MIN_START_GAP))
            start_gap_sum = 0
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                start_gap_sum += (row_state['l'+str(j)+'_start_gap']-MIN_START_GAP)/(MAX_START_GAP-MIN_START_GAP)
            row_state_list.append(start_gap_sum/FORMER_E_CNT_THRESHOLD)

            row_state_list.append((row_state['a_end_gap']-MIN_END_GAP)/(MAX_END_GAP-MIN_END_GAP))
            end_gap_sum = 0
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                end_gap_sum += (row_state['l'+str(j)+'_end_gap']-MIN_END_GAP)/(MAX_END_GAP-MIN_END_GAP)
            row_state_list.append(end_gap_sum/FORMER_E_CNT_THRESHOLD)

            row_state_list.append(row_state['worker_quality'])

            row_state_list.append((row_state['project_quality']-MIN_PROJECT_QUALITY)/(MAX_PROJECT_QUALITY-MIN_PROJECT_QUALITY))
            state_list.append(row_state_list)

            info_list.append([len(MDP_w[i]['state']),  # legal_max_action
                              row_state['a_category'],
                              row_state['a_industry'],
                              row_state['a_award'],
                              row_state['a_start_gap'],
                              row_state['a_end_gap'],
                              row_state['worker_quality'],
                              row_state['project_quality']])
        np_state = np.array(state_list)
        np_state = np.concatenate((np_state, np.zeros([MAX_AVAILABLE_ACTION - np_state.shape[0], np_state.shape[1]])), axis=0)
        batch_states.append(np_state)
        np_info = np.array(info_list)
        np_info = np.concatenate((np_info, np.zeros([MAX_AVAILABLE_ACTION - np_info.shape[0], np_info.shape[1]])), axis=0)
        batch_infos.append(np_info)

        # 【A】
        batch_actions.append(MDP_w[i]['action'])

        # 【R_w R_r】
        batch_rewards_w.append(MDP_w[i]['reward'])
        batch_rewards_r.append(MDP_r[i]['reward'])

        # 【Best_Info】
        batch_best_infos.append(MDP_w[i]['best_info'])

        # 【S'_w, Info'_w】
        next_state_list, next_info_list = [], []
        for next_row_state in MDP_w[i]['next_state']:
            next_row_state_list = []
            next_row_state_list.append(next_row_state['a_category'])
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                next_row_state_list.append(next_row_state['l'+str(j)+'_category'])

            next_row_state_list.append(next_row_state['a_industry'])
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                next_row_state_list.append(next_row_state['l'+str(j)+'_industry'])

            next_row_state_list.append((next_row_state['a_award']-MIN_AWARD)/(MAX_AWARD-MIN_AWARD))
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                next_row_state_list.append((next_row_state['l'+str(j)+'_award']-MIN_AWARD)/(MAX_AWARD-MIN_AWARD))
            
            next_row_state_list.append((next_row_state['a_start_gap']-MIN_START_GAP)/(MAX_START_GAP-MIN_START_GAP))
            start_gap_sum = 0
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                start_gap_sum += (next_row_state['l'+str(j)+'_start_gap']-MIN_START_GAP)/(MAX_START_GAP-MIN_START_GAP)
            next_row_state_list.append(start_gap_sum/FORMER_E_CNT_THRESHOLD)

            next_row_state_list.append((next_row_state['a_end_gap']-MIN_END_GAP)/(MAX_END_GAP-MIN_END_GAP))
            end_gap_sum = 0
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                end_gap_sum += (next_row_state['l'+str(j)+'_end_gap']-MIN_END_GAP)/(MAX_END_GAP-MIN_END_GAP)
            next_row_state_list.append(end_gap_sum/FORMER_E_CNT_THRESHOLD)

            next_row_state_list.append(next_row_state['worker_quality'])

            next_row_state_list.append((next_row_state['project_quality']-MIN_PROJECT_QUALITY)/(MAX_PROJECT_QUALITY-MIN_PROJECT_QUALITY))
            next_state_list.append(next_row_state_list)

            next_info_list.append([len(MDP_w[i]['next_state']),  # legal_max_action
                                  next_row_state['a_category'],
                                  next_row_state['a_industry'],
                                  next_row_state['a_award'],
                                  next_row_state['a_start_gap'],
                                  next_row_state['a_end_gap'],
                                  next_row_state['worker_quality'],
                                  next_row_state['project_quality']])
        np_next_state = np.array(next_state_list)
        np_next_state = np.concatenate((np_next_state, np.zeros([MAX_AVAILABLE_ACTION - np_next_state.shape[0], np_next_state.shape[1]])), axis=0)
        batch_next_states_w.append(np_next_state)
        np_next_info = np.array(next_info_list)
        np_next_info = np.concatenate((np_next_info, np.zeros([MAX_AVAILABLE_ACTION - np_next_info.shape[0], np_next_info.shape[1]])), axis=0)
        batch_next_infos_w.append(np_next_info)

        # 【S'_r, Info'_r】
        next_state_list, next_info_list = [], []
        for next_row_state in MDP_r[i]['next_state']:
            next_row_state_list = []
            next_row_state_list.append(next_row_state['a_category'])
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                next_row_state_list.append(next_row_state['l'+str(j)+'_category'])
            
            next_row_state_list.append(next_row_state['a_industry'])
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                next_row_state_list.append(next_row_state['l'+str(j)+'_industry'])
            
            next_row_state_list.append((next_row_state['a_award']-MIN_AWARD)/(MAX_AWARD-MIN_AWARD))
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                next_row_state_list.append((next_row_state['l'+str(j)+'_award']-MIN_AWARD)/(MAX_AWARD-MIN_AWARD))
            
            next_row_state_list.append((next_row_state['a_start_gap']-MIN_START_GAP)/(MAX_START_GAP-MIN_START_GAP))
            start_gap_sum = 0
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                start_gap_sum += (next_row_state['l'+str(j)+'_start_gap']-MIN_START_GAP)/(MAX_START_GAP-MIN_START_GAP)
            next_row_state_list.append(start_gap_sum/FORMER_E_CNT_THRESHOLD)

            next_row_state_list.append((next_row_state['a_end_gap']-MIN_END_GAP)/(MAX_END_GAP-MIN_END_GAP))
            end_gap_sum = 0
            for j in range(1, FORMER_E_CNT_THRESHOLD+1):
                end_gap_sum += (next_row_state['l'+str(j)+'_end_gap']-MIN_END_GAP)/(MAX_END_GAP-MIN_END_GAP)
            next_row_state_list.append(end_gap_sum/FORMER_E_CNT_THRESHOLD)

            next_row_state_list.append(next_row_state['worker_quality'])

            next_row_state_list.append((next_row_state['project_quality']-MIN_PROJECT_QUALITY)/(MAX_PROJECT_QUALITY-MIN_PROJECT_QUALITY))
            next_state_list.append(next_row_state_list)

            next_info_list.append([len(MDP_r[i]['next_state']),  # legal_max_action
                                  next_row_state['a_category'],
                                  next_row_state['a_industry'],
                                  next_row_state['a_award'],
                                  next_row_state['a_start_gap'],
                                  next_row_state['a_end_gap'],
                                  next_row_state['worker_quality'],
                                  next_row_state['project_quality']])
        np_next_state = np.array(next_state_list)
        np_next_state = np.concatenate((np_next_state, np.zeros([MAX_AVAILABLE_ACTION - np_next_state.shape[0], np_next_state.shape[1]])), axis=0)
        batch_next_states_r.append(np_next_state)
        np_next_info = np.array(next_info_list)
        np_next_info = np.concatenate((np_next_info, np.zeros([MAX_AVAILABLE_ACTION - np_next_info.shape[0], np_next_info.shape[1]])), axis=0)
        batch_next_infos_r.append(np_next_info)

    print(len(batch_states) == len(batch_actions) == len(batch_rewards_w) == len(batch_rewards_r) == \
          len(batch_next_states_w) == len(batch_next_states_r) == \
          len(batch_infos) == len(batch_best_infos) == len(batch_next_infos_w) == len(batch_next_infos_r))

    MDP_w_transition_filename = "../exp_data/MDP_transition_"+str(partition)+".pkl"
    with open(MDP_w_transition_filename, "wb") as f:
        pickle.dump({'states': np.array(batch_states),
                     'actions': np.array(batch_actions, dtype=np.int32),
                     'rewards_w': np.array(batch_rewards_w),
                     'rewards_r': np.array(batch_rewards_r),
                     'next_states_w': np.array(batch_next_states_w),
                     'next_states_r': np.array(batch_next_states_r),
                     'infos': np.array(batch_infos),
                     'best_infos': np.array(batch_best_infos),
                     'next_infos_w': np.array(batch_next_infos_w),
                     'next_infos_r': np.array(batch_next_infos_r)
                     }, f)
    
    del batch_states, batch_actions, batch_rewards_w, batch_rewards_r, batch_next_states_w, batch_next_states_r, batch_infos, batch_best_infos, batch_next_infos_w, batch_next_infos_r
    gc.collect()

处理分区0中……
处理记录0/2608543……
处理记录10000/2608543……
处理记录20000/2608543……
处理记录30000/2608543……
处理记录40000/2608543……
处理记录50000/2608543……
处理记录60000/2608543……
处理记录70000/2608543……
处理记录80000/2608543……
处理记录90000/2608543……
True
处理分区1中……
处理记录100000/2608543……
处理记录110000/2608543……
处理记录120000/2608543……
处理记录130000/2608543……
处理记录140000/2608543……
处理记录150000/2608543……
处理记录160000/2608543……
处理记录170000/2608543……
处理记录180000/2608543……
处理记录190000/2608543……
True
处理分区2中……
处理记录200000/2608543……
处理记录210000/2608543……
处理记录220000/2608543……
处理记录230000/2608543……
处理记录240000/2608543……
处理记录250000/2608543……
处理记录260000/2608543……
处理记录270000/2608543……
处理记录280000/2608543……
处理记录290000/2608543……
True
处理分区3中……
处理记录300000/2608543……
处理记录310000/2608543……
处理记录320000/2608543……
处理记录330000/2608543……
处理记录340000/2608543……
处理记录350000/2608543……
处理记录360000/2608543……
处理记录370000/2608543……
处理记录380000/2608543……
处理记录390000/2608543……
True
处理分区4中……
处理记录400000/2608543……
处理记录410000/2608543……
处理记录420000/2608543……
处理记录430000/2608543……
处理记录440000/2608543……
处理记录